# Heat Equation

![](heat_square.png)

*Figure 1*: Temperature field on the unit square with an internal uniform heat source
solved with homogeneous Dirichlet boundary conditions on the boundary.

## Introduction

The heat equation is the "Hello, world!" equation of finite elements.
Here we solve the equation on a unit square, with a uniform internal source.
The strong form of the (linear) heat equation is given by

$$
 -\nabla \cdot (k \nabla u) = f  \quad \textbf{x} \in \Omega,
$$

where $u$ is the unknown temperature field, $k$ the heat conductivity,
$f$ the heat source and $\Omega$ the domain. For simplicity we set $f = 1$
and $k = 1$. We will consider homogeneous Dirichlet boundary conditions such that
$$
u(\textbf{x}) = 0 \quad \textbf{x} \in \partial \Omega,
$$
where $\partial \Omega$ denotes the boundary of $\Omega$.
The resulting weak form is given given as follows: Find $u \in \mathbb{U}$ such that
$$
\int_{\Omega} \nabla \delta u \cdot \nabla u \ d\Omega = \int_{\Omega} \delta u \ d\Omega \quad \forall \delta u \in \mathbb{T},
$$
where $\delta u$ is a test function, and where $\mathbb{U}$ and $\mathbb{T}$ are suitable
trial and test function sets, respectively.

## Commented Program

Now we solve the problem in Ferrite. What follows is a program spliced with comments.

First we load Ferrite, and some other packages we need

In [1]:
using Ferrite, SparseArrays

We start by generating a simple grid with 20x20 quadrilateral elements
using `generate_grid`. The generator defaults to the unit square,
so we don't need to specify the corners of the domain.

In [2]:
grid = generate_grid(Quadrilateral, (20, 20));

### Trial and test functions
A `CellValues` facilitates the process of evaluating values and gradients of
test and trial functions (among other things). Since the problem
is a scalar problem we will use a `CellScalarValues` object. To define
this we need to specify an interpolation space for the shape functions.
We use Lagrange functions (both for interpolating the function and the geometry)
based on the two-dimensional reference "cube". We also define a quadrature rule based on
the same reference cube. We combine the interpolation and the quadrature rule
to a `CellScalarValues` object.

In [3]:
dim = 2
ip = Lagrange{dim, RefCube, 1}()
qr = QuadratureRule{dim, RefCube}(2)
cellvalues = CellScalarValues(qr, ip);

In [20]:
?CellScalarValues

search: CellScalarValues



```
CellScalarValues([::Type{T}], quad_rule::QuadratureRule, func_interpol::Interpolation, [geom_interpol::Interpolation])
CellVectorValues([::Type{T}], quad_rule::QuadratureRule, func_interpol::Interpolation, [geom_interpol::Interpolation])
```

A `CellValues` object facilitates the process of evaluating values of shape functions, gradients of shape functions, values of nodal functions, gradients and divergences of nodal functions etc. in the finite element cell. There are two different types of `CellValues`: `CellScalarValues` and `CellVectorValues`. As the names suggest, `CellScalarValues` utilizes scalar shape functions and `CellVectorValues` utilizes vectorial shape functions. For a scalar field, the `CellScalarValues` type should be used. For vector field, both subtypes can be used.

**Arguments:**

  * `T`: an optional argument (default to `Float64`) to determine the type the internal data is stored as.
  * `quad_rule`: an instance of a [`QuadratureRule`](@ref)
  * `func_interpol`: an instance of an [`Interpolation`](@ref) used to interpolate the approximated function
  * `geom_interpol`: an optional instance of a [`Interpolation`](@ref) which is used to interpolate the geometry

**Common methods:**

  * [`reinit!`](@ref)
  * [`getnquadpoints`](@ref)
  * [`getdetJdV`](@ref)
  * [`shape_value`](@ref)
  * [`shape_gradient`](@ref)
  * [`shape_symmetric_gradient`](@ref)
  * [`shape_divergence`](@ref)
  * [`function_value`](@ref)
  * [`function_gradient`](@ref)
  * [`function_symmetric_gradient`](@ref)
  * [`function_divergence`](@ref)
  * [`spatial_coordinate`](@ref)


In [17]:
?Lagrange

search: Lagrange DiscontinuousLagrange BubbleEnrichedLagrange



No documentation found.

# Summary

```
struct Lagrange{dim, shape, order}
```

# Supertype Hierarchy

```
Lagrange{dim, shape, order} <: Interpolation{dim, shape, order} <: Any
```


In [18]:
?Interpolation

search: Interpolation



```
Interpolation{ref_dim, ref_shape, order}()
```

Return an `Interpolation` on a `ref_dim`-dimensional reference shape (see [`AbstractRefShape`](@ref)) `ref_shape` and order `order`. `order` corresponds to the order of the interpolation. The interpolation is used to define shape functions to interpolate a function between nodes.

The following interpolations are implemented:

  * `Lagrange{1,RefCube,1}`
  * `Lagrange{1,RefCube,2}`
  * `Lagrange{2,RefCube,1}`
  * `Lagrange{2,RefCube,2}`
  * `Lagrange{2,RefTetrahedron,1}`
  * `Lagrange{2,RefTetrahedron,2}`
  * `Lagrange{2,RefTetrahedron,3}`
  * `Lagrange{2,RefTetrahedron,4}`
  * `Lagrange{2,RefTetrahedron,5}`
  * `BubbleEnrichedLagrange{2,RefTetrahedron,1}`
  * `CrouzeixRaviart{2,1}`
  * `Lagrange{3,RefCube,1}`
  * `Lagrange{3,RefCube,2}`
  * `Lagrange{3,RefTetrahedron,1}`
  * `Lagrange{3,RefTetrahedron,2}`
  * `Lagrange{3,RefPrism,1}`
  * `Lagrange{3,RefPrism,2}`
  * `Serendipity{2,RefCube,2}`
  * `Serendipity{3,RefCube,2}`

# Examples

```jldoctest
julia> ip = Lagrange{2,RefTetrahedron,2}()
Ferrite.Lagrange{2,Ferrite.RefTetrahedron,2}()

julia> getnbasefunctions(ip)
6
```


In [16]:
?QuadratureRule

search: QuadratureRule QuadraticQuadrilateral



```
QuadratureRule{dim,shape}([quad_rule_type::Symbol], order::Int)
```

Create a `QuadratureRule` used for integration. `dim` is the space dimension, `shape` an [`AbstractRefShape`](@ref) and `order` the order of the quadrature rule. `quad_rule_type` is an optional argument determining the type of quadrature rule, currently the `:legendre` and `:lobatto` rules are implemented.

A `QuadratureRule` is used to approximate an integral on a domain by a weighted sum of function values at specific points:

$\int\limits_\Omega f(\mathbf{x}) \text{d} \Omega \approx \sum\limits_{q = 1}^{n_q} f(\mathbf{x}_q) w_q$

The quadrature rule consists of $n_q$ points in space $\mathbf{x}_q$ with corresponding weights $w_q$.

In `Ferrite`, the `QuadratureRule` type is mostly used as one of the components to create a [`CellValues`](@ref) or [`FaceValues`](@ref) object.

**Common methods:**

  * [`getpoints`](@ref) : the points of the quadrature rule
  * [`getweights`](@ref) : the weights of the quadrature rule

**Example:**

```jldoctest
julia> QuadratureRule{2, RefTetrahedron}(1)
Ferrite.QuadratureRule{2,Ferrite.RefTetrahedron,Float64}([0.5], Tensors.Tensor{1,2,Float64,2}[[0.333333, 0.333333]])

julia> QuadratureRule{1, RefCube}(:lobatto, 2)
Ferrite.QuadratureRule{1,Ferrite.RefCube,Float64}([1.0, 1.0], Tensors.Tensor{1,1,Float64,1}[[-1.0], [1.0]])
```


In [5]:
?ip

search: ip zip Pipe pipeline PipeBuffer skip flipsign skipchars clipboard



No documentation found.

`ip` is of type `Lagrange{2, RefCube, 1}`.

# Summary

```
struct Lagrange{2, RefCube, 1}
```

# Supertype Hierarchy

```
Lagrange{2, RefCube, 1} <: Interpolation{2, RefCube, 1} <: Any
```


In [6]:
?qr

search: qr sqrt isqrt Quadrilateral QuadraticLine QuadratureRule Quadrilateral3D



No documentation found.

`qr` is of type `QuadratureRule{2, RefCube, Float64}`.

# Summary

```
struct QuadratureRule{2, RefCube, Float64}
```

# Fields

```
weights :: Vector{Float64}
points  :: Vector{Vec{2, Float64}}
```


### Degrees of freedom
Next we need to define a `DofHandler`, which will take care of numbering
and distribution of degrees of freedom for our approximated fields.
We create the `DofHandler` and then add a single scalar field called `:u`.
Lastly we `close!` the `DofHandler`, it is now that the dofs are distributed
for all the elements.

In [7]:
dh = DofHandler(grid)
add!(dh, :u, 1)
close!(dh);

Now that we have distributed all our dofs we can create our tangent matrix,
using `create_sparsity_pattern`. This function returns a sparse matrix
with the correct entries stored.

In [8]:
K = create_sparsity_pattern(dh)

441×441 SparseMatrixCSC{Float64, Int64} with 3721 stored entries:
⎡⠻⣦⡀⠀⢧⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠈⠻⣦⠈⢧⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠉⠓⠦⣄⡻⣮⡳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠙⢮⡻⣮⡳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠙⢮⡻⣮⡳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠙⢮⡻⢎⡳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢮⡻⣮⡳⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⣮⡻⣮⡳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢮⡻⣮⡳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢮⡻⣮⡳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢮⡻⣮⡳⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠹⢮⡻⣮⡳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢮⡻⣮⡳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢮⡻⣮⡳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢮⡱⣮⡳⣄⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢮⡻⣮⡳⣄⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢮⡻⣮⡳⣄⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢮⡻⣮⠳⣄⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡻⣮⡳⣄⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢮⠻⣦⎦

### Boundary conditions
In Ferrite constraints like Dirichlet boundary conditions
are handled by a `ConstraintHandler`.

In [9]:
ch = ConstraintHandler(dh);

Next we need to add constraints to `ch`. For this problem we define
homogeneous Dirichlet boundary conditions on the whole boundary, i.e.
the `union` of all the face sets on the boundary.

In [10]:
∂Ω = union(
    getfaceset(grid, "left"),
    getfaceset(grid, "right"),
    getfaceset(grid, "top"),
    getfaceset(grid, "bottom"),
);

Now we are set up to define our constraint. We specify which field
the condition is for, and our combined face set `∂Ω`. The last
argument is a function of the form $f(\textbf{x})$ or $f(\textbf{x}, t)$,
where $\textbf{x}$ is the spatial coordinate and
$t$ the current time, and returns the prescribed value. Since the boundary condition in
this case do not depend on time we define our function as $f(\textbf{x}) = 0$, i.e.
no matter what $\textbf{x}$ we return $0$. When we have
specified our constraint we `add!` it to `ch`.

In [11]:
dbc = Dirichlet(:u, ∂Ω, (x, t) -> 0)
add!(ch, dbc);

Finally we also need to `close!` our constraint handler. When we call `close!`
the dofs corresponding to our constraints are calculated and stored
in our `ch` object.

In [12]:
close!(ch)

ConstraintHandler:
  BCs:
    Field: u, Components: [1]

Note that if one or more of the constraints are time dependent we would use
`update!` to recompute prescribed values in each new timestep.

### Assembling the linear system

Now we have all the pieces needed to assemble the linear system, $K u = f$.
Assembling of the global system is done by looping over all the elements in order to
compute the element contributions $K_e$ and $f_e$, which are then assembled to the
appropriate place in the global $K$ and $f$.

#### Element assembly
We define the function `assemble_element!` (see below) which computes the contribution for
an element. The function takes pre-allocated `ke` and `fe` (they are allocated once and
then reused for all elements) so we first need to make sure that they are all zeroes at
the start of the function by using `fill!`. Then we loop over all the quadrature points,
and for each quadrature point we loop over all the (local) shape functions. We need the
value and gradient of the test function, `δu` and also the gradient of the trial function
`u`. We get all of these from `cellvalues`.

!!! note "Notation"
    Comparing with the brief finite element introduction in Introduction to FEM,
    the variables `δu`, `∇δu` and `∇u` are actually $\phi_i(\textbf{x}_q)$, $\nabla
    \phi_i(\textbf{x}_q)$ and $\nabla \phi_j(\textbf{x}_q)$, i.e. the evaluation of the
    trial and test functions in the quadrature point $\textbf{x}_q$. However, to
    underline the strong parallel between the weak form and the implementation, this
    example uses the symbols appearing in the weak form.

In [13]:
function assemble_element!(Ke::Matrix, fe::Vector, cellvalues::CellScalarValues)
    n_basefuncs = getnbasefunctions(cellvalues)
    # Reset to 0
    fill!(Ke, 0)
    fill!(fe, 0)
    # Loop over quadrature points
    for q_point in 1:getnquadpoints(cellvalues)
        # Get the quadrature weight
        dΩ = getdetJdV(cellvalues, q_point)
        # Loop over test shape functions
        for i in 1:n_basefuncs
            δu  = shape_value(cellvalues, q_point, i)
            ∇δu = shape_gradient(cellvalues, q_point, i)
            # Add contribution to fe
            fe[i] += δu * dΩ
            # Loop over trial shape functions
            for j in 1:n_basefuncs
                ∇u = shape_gradient(cellvalues, q_point, j)
                # Add contribution to Ke
                Ke[i, j] += (∇δu ⋅ ∇u) * dΩ
            end
        end
    end
    return Ke, fe
end

assemble_element! (generic function with 1 method)

In [19]:
?shape_gradient

search: shape_gradient shape_symmetric_gradient



```
shape_gradient(fe_v::Values, q_point::Int, base_function::Int)
```

Return the gradient of shape function `base_function` evaluated in quadrature point `q_point`.


#### Global assembly
We define the function `assemble_global` to loop over the elements and do the global
assembly. The function takes our `cellvalues`, the sparse matrix `K`, and our DofHandler
as input arguments and returns the assembled global stiffness matrix, and the assembled
global force vector. We start by allocating `Ke`, `fe`, and the global force vector `f`.
We also create an assembler by using `start_assemble`. The assembler lets us assemble into
`K` and `f` efficiently. We then start the loop over all the elements. In each loop
iteration we reinitialize `cellvalues` (to update derivatives of shape functions etc.),
compute the element contribution with `assemble_element!`, and then assemble into the
global `K` and `f` with `assemble!`.

!!! note "Notation"
    Comparing again with Introduction to FEM, `f` and `u` correspond to
    $\underline{\hat{f}}$ and $\underline{\hat{u}}$, since they represent the discretized
    versions. However, through the code we use `f` and `u` instead to reflect the strong
    connection between the weak form and the Ferrite implementation.

In [14]:
function assemble_global(cellvalues::CellScalarValues, K::SparseMatrixCSC, dh::DofHandler)
    # Allocate the element stiffness matrix and element force vector
    n_basefuncs = getnbasefunctions(cellvalues)
    Ke = zeros(n_basefuncs, n_basefuncs)
    fe = zeros(n_basefuncs)
    # Allocate global force vector f
    f = zeros(ndofs(dh))
    # Create an assembler
    assembler = start_assemble(K, f)
    # Loop over all cels
    for cell in CellIterator(dh)
        # Reinitialize cellvalues for this cell
        reinit!(cellvalues, cell)
        # Compute element contribution
        assemble_element!(Ke, fe, cellvalues)
        # Assemble Ke and fe into K and f
        assemble!(assembler, celldofs(cell), Ke, fe)
    end
    return K, f
end

assemble_global (generic function with 1 method)

### Solution of the system
The last step is to solve the system. First we call `assemble_global`
to obtain the global stiffness matrix `K` and force vector `f`.

In [15]:
cellvalues

CellScalarValues{2, Float64, RefCube} with 4 shape functions and 4 quadrature points

In [12]:
K, f = assemble_global(cellvalues, K, dh);

To account for the boundary conditions we use the `apply!` function.
This modifies elements in `K` and `f` respectively, such that
we can get the correct solution vector `u` by using `\`.

In [13]:
apply!(K, f, ch)
u = K \ f;

### Exporting to VTK
To visualize the result we export the grid and our field `u`
to a VTK-file, which can be viewed in e.g. [ParaView](https://www.paraview.org/).

In [14]:
vtk_grid("heat_equation", dh) do vtk
    vtk_point_data(vtk, dh, u)
end

1-element Vector{String}:
 "heat_equation.vtu"

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*